In [303]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import pandasql as ps
import os

In [ ]:
#Transfers fees

In [22]:
df = pd.read_excel(f"/Users/{os.environ['USER']}/Documents/Uni/PL_Players/Önlab 1/Scripts/5_transfer_modelling_datas/df_league_inv.xlsx")
df.head()

,Unnamed: 0,Unnamed: 0.1,Club,Name,Age,Nationality,Position,Pos,MarketValue,ClubInvolved,CountryInvolved,Fee,Movement,Season,Window,League,MainPosition,TransferType,LeagueInvolved
0,0,0,Shakhtar Donetsk,Bernard,20,Brazil,Left Winger,LW,10.0,Clube Atlético Mineiro,Brazil,25.0,In,2013,s,Ukrainian Premier League,Forward,Transfer,Brazil
1,1,1,Shakhtar Donetsk,Fred,20,Brazil,Central Midfield,CM,5.0,Sport Club Internacional,Brazil,15.0,In,2013,s,Ukrainian Premier League,Midfielder,Transfer,Brazil
2,2,2,Shakhtar Donetsk,Fernando,21,Brazil,Defensive Midfield,DM,10.0,Grêmio Foot-Ball Porto Alegrense,Brazil,11.0,In,2013,s,Ukrainian Premier League,Midfielder,Transfer,Brazil
3,3,3,Shakhtar Donetsk,Wellington Nem,21,Brazil,Right Winger,RW,8.0,Fluminense Football Club,Brazil,9.0,In,2013,s,Ukrainian Premier League,Forward,Transfer,Brazil
4,4,4,Shakhtar Donetsk,Facundo Ferreyra,22,Argentina,Centre-Forward,CF,6.0,Club Atlético Vélez Sarsfield,Argentina,7.0,In,2013,s,Ukrainian Premier League,Forward,Transfer,Argentina


In [385]:
predict_df = pd.read_excel(f"/Users/{os.environ['USER']}/Documents/Uni/PL_Players/Önlab 1/Scripts/5_transfer_modelling_datas/players_2021_predict_df4.xlsx")

In [23]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Club', 'Name', 'Age', 'Nationality',
       'Position', 'Pos', 'MarketValue', 'ClubInvolved', 'CountryInvolved',
       'Fee', 'Movement', 'Season', 'Window', 'League', 'MainPosition',
       'TransferType', 'LeagueInvolved'],
      dtype='object')

In [98]:
df = df.replace('China', 'China PR')
df = df.replace('Bosnia-Herzegovina', 'Bosnia Herzegovina')
df = df.replace('Guinea-Bissau', 'Guinea Bissau')
df = df.replace("Cote d'Ivoire", 'Ivory Coast')
df = df.replace('Ireland', 'Republic of Ireland')
df = df.replace('Korea, South', 'Korea DPR')
df = df.replace('Korea, South', 'Korea Republic')
df = df.replace('Palästina', 'Palestine')
df = df.replace('Trinidad and Tobago ', 'Trinidad & Tobago')

In [24]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175001 entries, 0 to 175000
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Club             175001 non-null  object 
 1   Name             175001 non-null  object 
 2   Age              175001 non-null  int64  
 3   Nationality      174969 non-null  object 
 4   Position         175001 non-null  object 
 5   Pos              175001 non-null  object 
 6   MarketValue      175001 non-null  float64
 7   ClubInvolved     157670 non-null  object 
 8   CountryInvolved  175001 non-null  object 
 9   Fee              175001 non-null  float64
 10  Movement         175001 non-null  object 
 11  Season           175001 non-null  int64  
 12  Window           175001 non-null  object 
 13  League           175001 non-null  object 
 14  MainPosition     175001 non-null  object 
 15  TransferType     175001 non-null  object 
 16  LeagueInvolved   175001 non-null  obje

In [26]:
df = df[df.TransferType == 'Transfer']
df = df[df.Movement == 'In']
df = df[~df.CountryInvolved.isin(['Without Club', 'Retired', 'Career break', 'Unknown', 'Ban'])]
df = df.drop(['Position', 'Movement', 'Window', 'TransferType'], axis=1)
df = df[df.Age > 10]
#exclude reserve teams
df = df[~df.ClubInvolved.str.contains('U1' ,na=False)]
df = df[~df.ClubInvolved.str.contains('U2' ,na=False)]
df = df[~df.ClubInvolved.str.contains('Reserves' ,na=False)]
df = df[~df.ClubInvolved.str.endswith(' B' ,na=False)]
df = df[~df.ClubInvolved.str.contains(' II' ,na=False)]
df = df[~df.ClubInvolved.str.contains(' Primavera' ,na=False)]

In [17]:
train_df = pd.read_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/fin_train_df.xlsx")
train_df.head()

,Unnamed: 0,NewName,MarketValue,ClubInvolved,League,MainPosition,LeagueInvolved,CorrMarketValue,ID,Name,...,Wage,Season,Years,DEGREE,Eigv,SP_LA_LIGA,EN_PL,IT_SERIE_A,FR_LIGUE_1,GE_BUNDESLIGA
0,0,Bernard,10.0,Clube Atlético Mineiro,Ukrainian Premier League,Forward,Brazil,5.0,205525,Bernard,...,€80K,2014,2,7,0.001684,0,0,0,0,0
1,1,Fred,5.0,Sport Club Internacional,Ukrainian Premier League,Midfielder,Brazil,2.5,209297,Fred,...,€50K,2014,2,0,0.000000,0,0,0,0,0
2,2,Fernando,10.0,Grêmio Foot-Ball Porto Alegrense,Ukrainian Premier League,Midfielder,Brazil,5.0,202642,Fernando,...,€60K,2014,2,3,0.000741,0,0,0,0,0
3,3,Fernando,10.0,Grêmio Foot-Ball Porto Alegrense,Ukrainian Premier League,Midfielder,Brazil,5.0,202642,Fernando,...,€60K,2014,2,3,0.000741,0,0,0,0,0
4,4,W. Nem,8.0,Fluminense Football Club,Ukrainian Premier League,Forward,Brazil,4.0,203279,Wellington Nem,...,€50K,2014,2,0,0.000000,0,0,0,0,0


In [18]:
train_df.columns

Index(['Unnamed: 0', 'NewName', 'MarketValue', 'ClubInvolved', 'League',
       'MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'ID', 'Name',
       'Age', 'Height_cm', 'Weight_kg', 'Country', 'Club', 'Joined',
       'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing',
       'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve',
       'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration',
       'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power',
       'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'Standing_tackle', 'Sliding_tackle', 'GK_diving',
       'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes', 'Value',
       'Wage', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL',
       'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA'],
      dtype='object')

In [20]:
train_df = train_df.drop(['Unnamed: 0'], axis = 1)

In [33]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17431 entries, 0 to 17430
Data columns (total 63 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NewName           17431 non-null  object 
 1   MarketValue       17431 non-null  float64
 2   ClubInvolved      17430 non-null  object 
 3   League            17431 non-null  object 
 4   MainPosition      17431 non-null  object 
 5   LeagueInvolved    17431 non-null  object 
 6   CorrMarketValue   17431 non-null  float64
 7   ID                17431 non-null  int64  
 8   Name              17431 non-null  object 
 9   Age               17431 non-null  int64  
 10  Height_cm         17431 non-null  object 
 11  Weight_kg         17431 non-null  object 
 12  Country           17431 non-null  object 
 13  Club              17431 non-null  object 
 14  Joined            12994 non-null  object 
 15  Position          17431 non-null  object 
 16  Foot              17431 non-null  object

In [96]:
#join tables with train df
q1 = """SELECT DF.*,
        m.Fee

        FROM train_df DF 

        LEFT JOIN df m
        ON DF.MarketValue = m.MarketValue
        AND DF.Age = m.Age
        AND DF.MainPosition = m.MainPosition
        AND DF.League = m.League
        AND DF.LeagueInvolved = m.LeagueInvolved
        AND DF.Season = m.Season+1
        AND DF.Country = m.Nationality
        """

train_fee_df = ps.sqldf(q1, locals())

In [97]:
train_fee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17982 entries, 0 to 17981
Data columns (total 64 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NewName           17982 non-null  object 
 1   MarketValue       17982 non-null  float64
 2   ClubInvolved      17981 non-null  object 
 3   League            17982 non-null  object 
 4   MainPosition      17982 non-null  object 
 5   LeagueInvolved    17982 non-null  object 
 6   CorrMarketValue   17982 non-null  float64
 7   ID                17982 non-null  int64  
 8   Name              17982 non-null  object 
 9   Age               17982 non-null  int64  
 10  Height_cm         17982 non-null  object 
 11  Weight_kg         17982 non-null  object 
 12  Country           17982 non-null  object 
 13  Club              17982 non-null  object 
 14  Joined            13299 non-null  object 
 15  Position          17982 non-null  object 
 16  Foot              17982 non-null  object

In [99]:
train_fee_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/fee_train_df.xlsx")

In [4]:
train_fee_d = pd.read_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/fee_train_df.xlsx")

In [6]:
train_fee_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17309 entries, 0 to 17308
Data columns (total 65 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        17309 non-null  int64  
 1   NewName           17309 non-null  object 
 2   MarketValue       17309 non-null  float64
 3   ClubInvolved      17308 non-null  object 
 4   League            17309 non-null  object 
 5   MainPosition      17309 non-null  object 
 6   LeagueInvolved    17309 non-null  object 
 7   CorrMarketValue   17309 non-null  float64
 8   ID                17309 non-null  int64  
 9   Name              17309 non-null  object 
 10  Age               17309 non-null  int64  
 11  Height_cm         17309 non-null  object 
 12  Weight_kg         17309 non-null  object 
 13  Country           17309 non-null  object 
 14  Club              17309 non-null  object 
 15  Joined            12897 non-null  object 
 16  Position          17309 non-null  object

Contract expiry

In [386]:
cwd = os.path.abspath(f"/Users/{os.envrion['USER']}/Documents/Uni/Szakdoga/Adatok/contract_expiry/") 
files = os.listdir(cwd) 

In [389]:
df_contract = pd.DataFrame()
for file in files:
     if file.endswith('.xlsx'):
         df_contract = df_contract.append(pd.read_excel(cwd + "/" + file), ignore_index=True) 
df_contract.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/roland.kovacs4ibm.com/Documents/Uni/Szakdoga/Adatok/contract_expiry/~$2021.xlsx'

In [413]:
df_contract = pd.read_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/contract_expiry/2021.xlsx")

In [414]:
df_contract['Contract'] = df_contract['Team & Contract'].str[-4:]

In [411]:
predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19126 entries, 0 to 19125
Data columns (total 58 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        19126 non-null  int64  
 1   Unnamed: 0.1      19126 non-null  int64  
 2   Name              19126 non-null  object 
 3   Age               19126 non-null  int64  
 4   Nationality       19126 non-null  object 
 5   Pos               19126 non-null  object 
 6   MarketValue       19126 non-null  float64
 7   LeagueInvolved    19126 non-null  object 
 8   Crossing          19126 non-null  int64  
 9   Finishing         19126 non-null  int64  
 10  Heading_accuracy  19126 non-null  int64  
 11  Short_passing     19126 non-null  int64  
 12  Volleys           19126 non-null  int64  
 13  Dribbling         19126 non-null  int64  
 14  Curve             19126 non-null  int64  
 15  FK_accuracy       19126 non-null  int64  
 16  Long_passing      19126 non-null  int64 

In [415]:
#join tables with train df
q1 = """SELECT DF.*,
        2022 as Season,
        m.ClubInvolved,
        m.Contract ContractGood

        FROM predict_df DF 

        LEFT JOIN df_contract m
        ON DF.ID = m.ID
        """

predict_contract_df = ps.sqldf(q1, locals())

predict_contract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19203 entries, 0 to 19202
Data columns (total 61 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        19203 non-null  int64  
 1   Unnamed: 0.1      19203 non-null  int64  
 2   Name              19203 non-null  object 
 3   Age               19203 non-null  int64  
 4   Nationality       19203 non-null  object 
 5   Pos               19203 non-null  object 
 6   MarketValue       19203 non-null  float64
 7   LeagueInvolved    19203 non-null  object 
 8   Crossing          19203 non-null  int64  
 9   Finishing         19203 non-null  int64  
 10  Heading_accuracy  19203 non-null  int64  
 11  Short_passing     19203 non-null  int64  
 12  Volleys           19203 non-null  int64  
 13  Dribbling         19203 non-null  int64  
 14  Curve             19203 non-null  int64  
 15  FK_accuracy       19203 non-null  int64  
 16  Long_passing      19203 non-null  int64 

In [416]:
predict_contract_df = predict_contract_df.drop_duplicates()

In [19]:
#join tables with train df
q1 = """SELECT DF.*,
        m.Contract

        FROM train_fee_d DF 

        LEFT JOIN df_contract m
        ON DF.Season = m.Season
        AND DF.ID = m.ID
        """

train_contract_df = ps.sqldf(q1, locals())

train_contract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17445 entries, 0 to 17444
Data columns (total 66 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        17445 non-null  int64  
 1   NewName           17445 non-null  object 
 2   MarketValue       17445 non-null  float64
 3   ClubInvolved      17444 non-null  object 
 4   League            17445 non-null  object 
 5   MainPosition      17445 non-null  object 
 6   LeagueInvolved    17445 non-null  object 
 7   CorrMarketValue   17445 non-null  float64
 8   ID                17445 non-null  int64  
 9   Name              17445 non-null  object 
 10  Age               17445 non-null  int64  
 11  Height_cm         17445 non-null  object 
 12  Weight_kg         17445 non-null  object 
 13  Country           17445 non-null  object 
 14  Club              17445 non-null  object 
 15  Joined            13019 non-null  object 
 16  Position          17445 non-null  object

In [417]:
predict_contract_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/contract_predict_df.xlsx")

In [420]:
predict_contract_df = pd.read_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/contract_predict_df.xlsx")

In [20]:
train_contract_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/contract_train_df.xlsx")

In [292]:
train_contract_df = pd.read_excel((f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/contract_train_df.xlsx"))
train_contract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17245 entries, 0 to 17244
Data columns (total 66 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        17245 non-null  int64  
 1   NewName           17245 non-null  object 
 2   MarketValue       17245 non-null  float64
 3   ClubInvolved      17245 non-null  object 
 4   League            17245 non-null  object 
 5   MainPosition      17245 non-null  object 
 6   LeagueInvolved    17245 non-null  object 
 7   CorrMarketValue   17245 non-null  float64
 8   ID                17245 non-null  int64  
 9   Name              17245 non-null  object 
 10  Age               17245 non-null  int64  
 11  Height_cm         17245 non-null  object 
 12  Weight_kg         17245 non-null  object 
 13  Country           17245 non-null  object 
 14  Club              17245 non-null  object 
 15  Joined            12856 non-null  object 
 16  Position          17245 non-null  object

Fbref

In [313]:
def season_add(row):
   if row['Season'] == '20/21' :
      return '2022'
   if row['Season'] == '19/20' :
      return '2021'
   if row['Season'] == '18/19' :
      return '2020'
   if row['Season'] == '17/18':
      return '2019'
   if row['Season'] == '16/17':
      return '2018'
   if row['Season'] == '15/16':
      return '2017'
   if row['Season'] == '14/15':
      return '2016'
   if row['Season'] == '13/14':
      return '2015'
   if row['Season'] == '12/13':
      return '2014'
   if row['Season'] == '11/12':
      return '2013'
   if row['Season'] == '10/11':
      return '2012'
   if row['Season'] == '09/10':
      return '2011'
   if row['Season'] == '08/09':
      return '2010'
   if row['Season'] == '07/08':
      return '2009'
   if row['Season'] == '06/07':
      return '2008'

In [314]:
cwd = os.path.abspath(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Fbref_data/")
folders = os.listdir(cwd) 
folders.remove('.DS_Store') 

In [315]:
league_df = pd.DataFrame()
standard_df = pd.DataFrame()
shooting_df = pd.DataFrame()
gk_df = pd.DataFrame()
player_standard_df = pd.DataFrame()
player_shooting_df = pd.DataFrame()
player_gk_df = pd.DataFrame()


num = 1
for folder in folders:
    cwd = os.path.abspath(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/Fbref_data/{folder}/")
    files = os.listdir(cwd) 
    for file in files:
        if file.endswith('league2.xlsx'):
            league_df = league_df.append(pd.read_excel(cwd + "/" + file), ignore_index=True) 
        if file.endswith('squad_standatd_data2.xlsx'):
            standard_df = standard_df.append(pd.read_excel(cwd + "/" + file), ignore_index=True) 
        if file == 'shooting2.xlsx':
            shooting_df = shooting_df.append(pd.read_excel(cwd + "/" + file), ignore_index=True) 
        if file.endswith('gk2.xlsx'):
            gk_df = gk_df.append(pd.read_excel(cwd + "/" + file), ignore_index=True) 
        if file.endswith('player_standard2.xlsx'):
            player_standard_df = player_standard_df.append(pd.read_excel(cwd + "/" + file), ignore_index=True) 
        if file.endswith('player_shooting2.xlsx'):
            player_shooting_df = player_shooting_df.append(pd.read_excel(cwd + "/" + file), ignore_index=True) 
        if file.endswith('player_goalkeeping2.xlsx'):
            player_gk_df = player_gk_df.append(pd.read_excel(cwd + "/" + file), ignore_index=True) 
    print(f'{num}/{len(folders)}')
    num += 1

1/33
2/33
3/33
4/33
5/33
6/33
7/33
8/33
9/33
10/33
11/33
12/33
13/33
14/33
15/33
16/33
17/33
18/33
19/33
20/33
21/33
22/33
23/33
24/33
25/33
26/33
27/33
28/33
29/33
30/33
31/33
32/33
33/33


In [316]:
league_df['Year'] = league_df.apply(lambda row: season_add(row), axis=1)
standard_df['Year'] = standard_df.apply(lambda row: season_add(row), axis=1)
shooting_df['Year'] = shooting_df.apply(lambda row: season_add(row), axis=1)
gk_df['Year'] = gk_df.apply(lambda row: season_add(row), axis=1)
player_standard_df['Year'] = player_standard_df.apply(lambda row: season_add(row), axis=1)
player_shooting_df['Year'] = player_shooting_df.apply(lambda row: season_add(row), axis=1)
player_gk_df['Year'] = player_gk_df.apply(lambda row: season_add(row), axis=1)

In [317]:
player_standard_df['NewName'] = None
player_standard_df['LastName'] = None
player_standard_df['FirstLastName'] = None
for i, r in player_standard_df.iterrows():
    name = player_standard_df.at[i,'Player']
    try:
        if ' ' in name:
            str_pos = name.find(' ')
            first_char = name[0]
            new_name = first_char + '. ' + name[str_pos+1:]
            player_standard_df.at[i,'NewName'] = new_name

            end_pos = name.rfind(' ')
            player_standard_df.at[i,'LastName'] = name[end_pos+1:]

            player_standard_df.at[i,'FirstLastName'] = first_char + '. ' + name[end_pos+1:]
        else:
            player_standard_df.at[i,'NewName'] = player_standard_df.at[i,'Player']
            player_standard_df.at[i,'LastName'] = player_standard_df.at[i,'Player']
            player_standard_df.at[i,'FirstLastName'] = player_standard_df.at[i,'Player']
    except:
        pass

In [318]:
player_shooting_df['NewName'] = None
player_shooting_df['LastName'] = None
player_shooting_df['FirstLastName'] = None
for i, r in player_shooting_df.iterrows():
    name = player_shooting_df.at[i,'Player']
    try:
        if ' ' in name:
            str_pos = name.find(' ')
            first_char = name[0]
            new_name = first_char + '. ' + name[str_pos+1:]
            player_shooting_df.at[i,'NewName'] = new_name

            end_pos = name.rfind(' ')
            player_shooting_df.at[i,'LastName'] = name[end_pos+1:]

            player_shooting_df.at[i,'FirstLastName'] = first_char + '. ' + name[end_pos+1:]
        else:
            player_shooting_df.at[i,'NewName'] = player_shooting_df.at[i,'Player']
            player_shooting_df.at[i,'LastName'] = player_shooting_df.at[i,'Player']
            player_shooting_df.at[i,'FirstLastName'] = player_shooting_df.at[i,'Player']
    except:
        pass

In [319]:
player_gk_df['NewName'] = None
player_gk_df['LastName'] = None
player_gk_df['FirstLastName'] = None
for i, r in player_gk_df.iterrows():
    name = player_gk_df.at[i,'Player']
    try:
        if ' ' in name:
            str_pos = name.find(' ')
            first_char = name[0]
            new_name = first_char + '. ' + name[str_pos+1:]
            player_gk_df.at[i,'NewName'] = new_name

            end_pos = name.rfind(' ')
            player_gk_df.at[i,'LastName'] = name[end_pos+1:]

            player_gk_df.at[i,'FirstLastName'] = first_char + '. ' + name[end_pos+1:]
        else:
            player_gk_df.at[i,'NewName'] = player_gk_df.at[i,'Player']
            player_gk_df.at[i,'LastName'] = player_gk_df.at[i,'Player']
            player_gk_df.at[i,'FirstLastName'] = player_gk_df.at[i,'Player']
    except:
        pass

In [301]:
#join tables with train df
q1 = """SELECT DF.*,
        m.Rk as ClubInvolvedRank,
        m.W as ClubInvolvedWin,
        m.D as ClubInvolvedDraw,
        m.L as ClubInvolvedLoose,
        m.GF as ClubInvolvedGoalFor,
        m.GA as ClubInvolvedGoalAgainst,
        m.Pts as ClubInvolvedPoints,
        b.Rk as ClubRank,
        b.W as ClubWin,
        b.D as ClubDraw,
        b.L as ClubLoose,
        b.GF as ClubGoalFor,
        b.GA as ClubGoalAgainst,
        b.Pts as ClubPoint,
        c.Age as ClubInvolvedAge,
        c.Poss as ClubInovlvedPossession,
        c.Ast as ClubInvolvedAssists,
        c."G-PK" as ClubInvolvedNonPenaltyGoal,
        c.CrdY as ClubInvolvedYellowCard,
        c.CrdR as ClubInvolvedRedCard,
        d.Age as ClubAge,
        d.Poss as ClubPossession,
        d.Ast as ClubAssists,
        d."G-PK" as ClubNonPenaltyGoal,
        d.CrdY as ClubYellowCard,
        d.CrdR as ClubRedCard,
        e."# Pl" as ClubInvolvedNumPlayersUsed,
        e.Sh as ClubInvolvedShots,
        e.SoT as ClubInvolvedShotsOnTarget,
        e."SoT%" as ClubInvolvedShotsOnTargetRate,
        e."Sh/90" as ClubInvolvedShotsPer90,
        e."SoT/90" as ClubInvolvedShotsOnTargetPer90,
        e."G/Sh" as ClubInvolvedGoalPerShot,
        e."G/SoT" as ClubInvolvedGoalPerShotOnTarget,
        f."# Pl" as ClubNumPlayersUsed,
        f.Sh as ClubShots,
        f.SoT as ClubShotsOnTarget,
        f."SoT%" as ClubShotsOnTargetRate,
        f."Sh/90" as ClubShotsPer90,
        f."SoT/90" as ClubShotsOnTargetPer90,
        f."G/Sh" as ClubGoalPerShot,
        f."G/SoT" as ClubGoalPerShotOnTarget,
        g.GA90 as ClubInvolvedGoalAgainstPer90,
        g.SoTA as ClubInvolvedShotOnTargetAgainst,
        g.Saves as ClubInvolvedSaves,
        g."Save%" as ClubInvolvedSaveRate,
        g.CS as ClubInvolvedCleanSheets,
        g."CS%" as ClubInvolvedCleanSheetsRate,
        h.GA90 as ClubGoalAgainstPer90,
        h.SoTA as ClubShotOnTargetAgainst,
        h.Saves as ClubSaves,
        h."Save%" as ClubSaveRate,
        h.CS as ClubCleanSheets,
        h."CS%" as ClubCleanSheetsRate,
        m.Squad,
        b.Squad

        FROM train_contract_df DF 

        LEFT JOIN league_df m
        ON DF.Season = m.Year
        AND DF.ClubInvolved LIKE '%' || m.Squad || '%'
        AND DF.LeagueInvolved = m.League

        LEFT JOIN league_df b
        ON DF.Season = b.Year
        AND DF.Club LIKE '%' || b.Squad || '%'
        AND DF.League = b.League

        LEFT JOIN standard_df c
        ON DF.Season = c.Year
        AND DF.ClubInvolved LIKE '%' || c.Squad || '%'
        AND DF.LeagueInvolved = c.League

        LEFT JOIN standard_df d
        ON DF.Season = d.Year
        AND DF.Club LIKE '%' || d.Squad || '%'
        AND DF.League = d.League

        LEFT JOIN shooting_df e
        ON DF.Season = e.Year
        AND DF.ClubInvolved LIKE '%' || e.Squad || '%'
        AND DF.LeagueInvolved = e.League

        LEFT JOIN shooting_df f
        ON DF.Season = f.Year
        AND DF.Club LIKE '%' || f.Squad || '%'
        AND DF.League = f.League

        LEFT JOIN gk_df g
        ON DF.Season = g.Year
        AND DF.ClubInvolved LIKE '%' || g.Squad || '%'
        AND DF.LeagueInvolved = g.League

        LEFT JOIN gk_df h
        ON DF.Season = h.Year
        AND DF.Club LIKE '%' || h.Squad || '%'
        AND DF.League = h.League
        """

train_df_temp = ps.sqldf(q1, locals())

train_df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17245 entries, 0 to 17244
Columns: 122 entries, Unnamed: 0 to Squad
dtypes: float64(68), int64(36), object(18)
memory usage: 16.1+ MB


In [421]:
predict_contract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13093 entries, 0 to 13092
Data columns (total 60 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              13093 non-null  object 
 1   Age               13093 non-null  int64  
 2   Nationality       13093 non-null  object 
 3   Pos               13093 non-null  object 
 4   MarketValue       13093 non-null  float64
 5   LeagueInvolved    13093 non-null  object 
 6   Crossing          13093 non-null  int64  
 7   Finishing         13093 non-null  int64  
 8   Heading_accuracy  13093 non-null  int64  
 9   Short_passing     13093 non-null  int64  
 10  Volleys           13093 non-null  int64  
 11  Dribbling         13093 non-null  int64  
 12  Curve             13093 non-null  int64  
 13  FK_accuracy       13093 non-null  int64  
 14  Long_passing      13093 non-null  int64  
 15  Ball_control      13093 non-null  int64  
 16  Acceleration      13093 non-null  int64 

In [435]:
predict_contract_df = predict_contract_df.replace('Atlético de Madrid', 'Atlético Madrid', regex=True)
predict_contract_df = predict_contract_df.replace('Real Sporting de Gijón Jun 30, ', 'Real Sporting de Gijón', regex=True)
predict_contract_df = predict_contract_df.replace('Atlético de Madrid Jun 30, ', 'Atlético Madrid', regex=True)
predict_contract_df = predict_contract_df.replace('Málaga CF', 'Málaga', regex=True)
predict_contract_df = predict_contract_df.replace('Cádiz CF Jun 30, ', 'Cádiz', regex=True)

In [436]:
#join tables with train df
q1 = """SELECT DF.*,
        m.Rk as ClubInvolvedRank,
        m.W as ClubInvolvedWin,
        m.D as ClubInvolvedDraw,
        m.L as ClubInvolvedLoose,
        m.GF as ClubInvolvedGoalFor,
        m.GA as ClubInvolvedGoalAgainst,
        m.Pts as ClubInvolvedPoints,
        c.Age as ClubInvolvedAge,
        c.Poss as ClubInovlvedPossession,
        c.Ast as ClubInvolvedAssists,
        c."G-PK" as ClubInvolvedNonPenaltyGoal,
        c.CrdY as ClubInvolvedYellowCard,
        c.CrdR as ClubInvolvedRedCard,
        e."# Pl" as ClubInvolvedNumPlayersUsed,
        e.Sh as ClubInvolvedShots,
        e.SoT as ClubInvolvedShotsOnTarget,
        e."SoT%" as ClubInvolvedShotsOnTargetRate,
        e."Sh/90" as ClubInvolvedShotsPer90,
        e."SoT/90" as ClubInvolvedShotsOnTargetPer90,
        e."G/Sh" as ClubInvolvedGoalPerShot,
        e."G/SoT" as ClubInvolvedGoalPerShotOnTarget,
        g.GA90 as ClubInvolvedGoalAgainstPer90,
        g.SoTA as ClubInvolvedShotOnTargetAgainst,
        g.Saves as ClubInvolvedSaves,
        g."Save%" as ClubInvolvedSaveRate,
        g.CS as ClubInvolvedCleanSheets,
        g."CS%" as ClubInvolvedCleanSheetsRate

        FROM predict_contract_df DF 

        LEFT JOIN league_df m
        ON DF.Season = m.Year
        AND DF.ClubInvolved LIKE '%' || m.Squad || '%'
        AND DF.LeagueInvolved = m.League

        LEFT JOIN standard_df c
        ON DF.Season = c.Year
        AND DF.ClubInvolved LIKE '%' || c.Squad || '%'
        AND DF.LeagueInvolved = c.League

        LEFT JOIN shooting_df e
        ON DF.Season = e.Year
        AND DF.ClubInvolved LIKE '%' || e.Squad || '%'
        AND DF.LeagueInvolved = e.League

        LEFT JOIN gk_df g
        ON DF.Season = g.Year
        AND DF.ClubInvolved LIKE '%' || g.Squad || '%'
        AND DF.LeagueInvolved = g.League

        """

predict_df_temp = ps.sqldf(q1, locals())

predict_df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13093 entries, 0 to 13092
Data columns (total 87 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Name                             13093 non-null  object 
 1   Age                              13093 non-null  int64  
 2   Nationality                      13093 non-null  object 
 3   Pos                              13093 non-null  object 
 4   MarketValue                      13093 non-null  float64
 5   LeagueInvolved                   13093 non-null  object 
 6   Crossing                         13093 non-null  int64  
 7   Finishing                        13093 non-null  int64  
 8   Heading_accuracy                 13093 non-null  int64  
 9   Short_passing                    13093 non-null  int64  
 10  Volleys                          13093 non-null  int64  
 11  Dribbling                        13093 non-null  int64  
 12  Curve             

In [437]:
print(predict_df_temp.notnull().sum()[-50:])

GK_kicking                         13093
GK_positioning                     13093
GK_reflexes                        13093
ID                                 13093
Joined                             13093
IntReputation                      13093
Foot                               13093
Position                           13093
Country                            13093
NewMarketValue                     13093
DEGREE                             13093
Eigv                               13093
SP_LA_LIGA                         13093
EN_PL                              13093
IT_SERIE_A                         13093
FR_LIGUE_1                         13093
GE_BUNDESLIGA                      13093
Years                              13093
MainPosition                       13093
Season                             13093
ClubInvolved                       13014
ContractGood                       13093
ContractLeft                       12076
ClubInvolvedRank                    9758
ClubInvolvedWin 

In [438]:
predict_df_temp.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/predict_df_temp.xlsx")

In [395]:
print(set(league_df.League.values))

{nan, 'Croatian Prva HNL', 'Holland Eredivisie', 'Ukrainian Premier League', 'Czech Republic Gambrinus Liga', 'Denmark', 'Brazil', 'German 2 Bundesliga', 'Swedish Allsvenskan', 'Turkish Süper Lig', 'Russian Premier League', 'Greek Super League', 'German 1 Bundesliga', 'England', 'Swiss Super League', 'Austria', 'Italian Serie A', 'Spain Primera Division', 'English League Championship', 'Spanish Segunda División', 'Scottish Premiership', 'Italian Serie B', 'Chile', 'Portuguese Liga ZON SAGRES', 'French Ligue 2', 'Argentina', 'English Premier League', 'Belgium', 'French Ligue 1'}


In [396]:
print(set(predict_contract_df.LeagueInvolved.values))

{'Mexico', 'Croatian Prva HNL', 'Korea, South', 'Holland Eredivisie', 'Ukrainian Premier League', 'Czech Republic Gambrinus Liga', 'Kosovo', 'Slovenia', "Cote d'Ivoire", 'Panama', 'Romanian Liga I', 'United Arab Emirates', 'Austrian Football Bundesliga', 'Saudi Arabia', 'Brazil', 'German 2 Bundesliga', 'Belgian Jupiler Pro League', 'Polish T-Mobile Ekstraklasa', 'Swedish Allsvenskan', 'Turkish Süper Lig', 'Egypt', 'China', 'Hungary', 'Russian Premier League', 'Greek Super League', 'German 1 Bundesliga', 'Russia', 'Swiss Super League', 'Rep Ireland Airtricity League', 'Uruguay', 'England', 'Poland', 'New Zealand', 'Austria', 'United States', 'Bolivia', 'Spain', 'Italian Serie A', 'France', 'Ecuador', 'Bulgaria', 'Spain Primera Division', 'Colombia', 'English League Championship', 'India', 'Japan', 'Spanish Segunda División', 'Cameroon', 'Northern Ireland', 'Venezuela', 'Norwegian Eliteserien', 'Scottish Premiership', 'Italian Serie B', 'Chile', 'Netherlands', 'Portuguese Liga ZON SAGRES

In [431]:
clubs = list(set(predict_df_temp[predict_df_temp.LeagueInvolved == 'Austrian Football Bundesliga'].ClubInvolved.values))
clubs = [x for x in clubs if x != None]
clubs = sorted(clubs)
print(clubs)

['FC Flyeralarm Admira', 'FC Flyeralarm Admira Jun 30, ', 'FC Red Bull Salzburg', 'FC Red Bull Salzburg Jun 30, ', 'FK Austria Wien', 'FK Austria Wien Jun 30, ', 'LASK Linz', 'LASK Linz Jun 30, ', 'RZ Pellets Wolfsberger AC', 'RZ Pellets Wolfsberger AC Jun 30, ', 'SC Rheindorf Altach', 'SK Rapid Wien', 'SK Rapid Wien Jun 30, ', 'SK Sturm Graz', 'SK Sturm Graz Jun 30, ', 'SV Ried', 'TSV Egger Glas Hartberg', 'TSV Egger Glas Hartberg Jun 30, ', 'WSG Tirol', 'WSG Tirol Jun 30, ', 'spusu SKN St. Pölten', 'spusu SKN St. Pölten Jun 30, ']


In [434]:
print(set(league_df[league_df.League == 'Austria'].Squad.values))

{nan, 'SKN St. Pölten', 'SK Rapid Wien', 'Wacker Tirol', 'WSG Tirol', 'FC Wacker Innsbruck', 'Grazer AK', 'LASK Linz', 'TSV Hartberg', 'SV Grödig', 'Wolfsberger AC', 'Austria Kärnten', 'SC Magna Wiener Neustadt', 'SCR Altach Juniors', 'SV Kapfenberg', 'SK Sturm Graz', 'Red Bull Salzburg', 'Mattersburg', 'FC Admira Wacker Mödling', 'ASKÖ Pasching', 'LASK', 'Austria Vienna', 'SV Ried'}


In [ ]:
predict_contract_df = predict_contract_df.replace('Atlético de Madrid', 'Atlético Madrid', regex=True)
predict_contract_df = predict_contract_df.replace('Real Sporting de Gijón Jun 30, ', 'Real Sporting de Gijón', regex=True)
predict_contract_df = predict_contract_df.replace('Atlético de Madrid Jun 30, ', 'Atlético Madrid', regex=True)
predict_contract_df = predict_contract_df.replace('Málaga CF', 'Málaga', regex=True)
predict_contract_df = predict_contract_df.replace('Cádiz CF Jun 30, ', 'Cádiz', regex=True)

In [ ]:
outf_train = outf_train.drop(['ClubInvolvedPlayed', 'ClubInvolvedWin', 'ClubInvolvedDraw',\
     'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints',\
          'ClubPlayed', 'ClubWin', 'ClubDraw', 'ClubLoose', 'ClubGoalFor', 'ClubGoalAgainst', 'ClubPoint',\
               'ClubInvolvedAssists', 'ClubInvolvedNonPenaltyGoal', 'ClubInvolvedYellowCard', 'ClubInvolvedRedCard',\
                    'ClubAssists', 'ClubNonPenaltyGoal', 'ClubYellowCard', 'ClubRedCard', 'ClubInvolvedNumPlayersUsed',\
                         'ClubInvolvedShotsOnTarget', 'ClubInvolvedGoalPerShotOnTarget', 'ClubNumPlayersUsed', 'ClubShotsOnTarget',\
                         'ClubGoalPerShotOnTarget', 'ClubInvolvedShotOnTargetAgainst', 'ClubInvolvedCleanSheets', 'ClubShotOnTargetAgainst',\
                              'ClubCleanSheets', 'MatchesPlayed', 'Starts', 'MatchesPlayed', 'Goals', 'Assists', 'NonPenaltyGoals', 'YellowCards',\
                                   'RedCards', 'ShotsOnTarget',  'GoalsPer90', 'AssistsPer90', 'GoalsAssistsPer90', 'ShotsOnTargetPer90'], axis=1)

In [ ]:
outf_train = outf_train.drop(['ClubInvolvedPlayed', 'ClubInvolvedWin', 'ClubInvolvedDraw',\
     'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints',\
          'ClubPlayed', 'ClubWin', 'ClubDraw', 'ClubLoose', 'ClubGoalFor', 'ClubGoalAgainst', 'ClubPoint',\
               'ClubInvolvedAssists', 'ClubInvolvedNonPenaltyGoal', 'ClubInvolvedYellowCard', 'ClubInvolvedRedCard',\
                    'ClubAssists', 'ClubNonPenaltyGoal', 'ClubYellowCard', 'ClubRedCard', 'ClubInvolvedNumPlayersUsed',\
                         'ClubInvolvedShotsOnTarget', 'ClubInvolvedGoalPerShotOnTarget', 'ClubNumPlayersUsed', 'ClubShotsOnTarget',\
                         'ClubGoalPerShotOnTarget', 'ClubInvolvedShotOnTargetAgainst', 'ClubInvolvedCleanSheets', 'ClubShotOnTargetAgainst',\
                              'ClubCleanSheets', 'MatchesPlayed', 'Starts', 'MatchesPlayed', 'Goals', 'Assists', 'NonPenaltyGoals', 'YellowCards',\
                                   'RedCards', 'ShotsOnTarget',  'GoalsPer90', 'AssistsPer90', 'GoalsAssistsPer90', 'ShotsOnTargetPer90'], axis=1)

In [303]:
train_df_temp.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_df_temp.xlsx")

In [257]:
train_df_ws = pd.read_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_ws_df.xlsx")

In [275]:
train_df_ws.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17463 entries, 0 to 17462
Columns: 234 entries, Unnamed: 0 to assist
dtypes: float64(170), int64(35), object(29)
memory usage: 31.2+ MB


In [274]:
player_standard_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Rk', 'Player', 'Nation', 'Pos', 'Squad',
       'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'Gls/90', 'Ast/90', 'G+A/90', 'G-PK/90',
       'G+A-PK', 'Season', 'League', 'Leaegue', 'Year', 'NewName', 'LastName',
       'FirstLastName'],
      dtype='object')

In [277]:
set(player_standard_df.Pos)

{'DF',
 'DF,FW',
 'DF,GK',
 'DF,MF',
 'FW',
 'FW,DF',
 'FW,GK',
 'FW,MF',
 'GK',
 'GK,DF',
 'GK,FW',
 'GK,MF',
 'MF',
 'MF,DF',
 'MF,FW',
 'MF,GK',
 'Pos',
 nan}

In [278]:
#join tables with train df
q1 = """SELECT DF.*,
        i.MP as MatchesPlayed,
        i.Starts,
        i.Min as MinutesPlayed,
        i."90s" as MinutesPlayedPer90,
        i.Gls as Goals,
        i.Ast as Assists,
        i."G-PK" as NonPenaltyGoals,
        i.CrdY as YellowCards,
        i.CrdR as RedCards,
        i."Gls/90" as GoalsPer90,
        i."Ast/90" as AssistsPer90,
        i."G+A/90" as GoalsAssistsPer90,
        i.Player,
        i.Pos,
        i.Nation,
        i.Squad FbrefClub


        FROM train_df_ws DF 

        LEFT JOIN player_standard_df i
        ON DF.Season = i.Year
        AND DF.ClubInvolved LIKE '%' || i.Squad || '%'
        AND (i.NEWNAME = DF.NewNAME
        OR i.Player = DF.newNAME
        OR i.LASTNAME = DF.newNAME
        OR i.FIRSTLASTNAME = DF.newNAME)
        AND i.Pos != 'GK'
        """

train_df_temp2 = ps.sqldf(q1, locals())

train_df_temp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17539 entries, 0 to 17538
Columns: 250 entries, Unnamed: 0 to FbrefClub
dtypes: float64(170), int64(35), object(45)
memory usage: 33.5+ MB


In [440]:
#join tables with train df
q1 = """SELECT DF.*,
        i.MP as MatchesPlayed,
        i.Starts,
        i.Min as MinutesPlayed,
        i."90s" as MinutesPlayedPer90,
        i.Gls as Goals,
        i.Ast as Assists,
        i."G-PK" as NonPenaltyGoals,
        i.CrdY as YellowCards,
        i.CrdR as RedCards,
        i."Gls/90" as GoalsPer90,
        i."Ast/90" as AssistsPer90,
        i."G+A/90" as GoalsAssistsPer90,
        i.Player,
        i.Pos,
        i.Nation,
        i.Squad FbrefClub


        FROM predict_df_temp DF 

        LEFT JOIN player_standard_df i
        ON DF.Season = i.Year
        AND DF.ClubInvolved LIKE '%' || i.Squad || '%'
        AND (i.NEWNAME = DF.Name
        OR i.Player = DF.Name
        OR i.LASTNAME = DF.Name
        OR i.FIRSTLASTNAME = DF.Name)
        AND i.Pos != 'GK'
        """

predict_df_temp2 = ps.sqldf(q1, locals())

predict_df_temp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13102 entries, 0 to 13101
Columns: 103 entries, Name to FbrefClub
dtypes: float64(31), int64(46), object(26)
memory usage: 10.3+ MB


In [441]:
print(predict_df_temp2.notnull().sum()[-30:])

ClubInvolvedNumPlayersUsed         9545
ClubInvolvedShots                  9545
ClubInvolvedShotsOnTarget          9545
ClubInvolvedShotsOnTargetRate      9545
ClubInvolvedShotsPer90             9545
ClubInvolvedShotsOnTargetPer90     9545
ClubInvolvedGoalPerShot            9545
ClubInvolvedGoalPerShotOnTarget    9545
ClubInvolvedGoalAgainstPer90       9517
ClubInvolvedShotOnTargetAgainst    9517
ClubInvolvedSaves                  9517
ClubInvolvedSaveRate               9517
ClubInvolvedCleanSheets            9738
ClubInvolvedCleanSheetsRate        9738
MatchesPlayed                      6039
Starts                             6039
MinutesPlayed                      6039
MinutesPlayedPer90                 6039
Goals                              6039
Assists                            6031
NonPenaltyGoals                    6039
YellowCards                        6039
RedCards                           6039
GoalsPer90                         6039
AssistsPer90                       6031


In [442]:
predict_df_temp2.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/predict_df_temp2.csv")

In [444]:
#join tables with train df
q1 = """SELECT DF.*,
        j.Sh as Shots,
        j.SoT as ShotsOnTarget,
        j."SoT%" as ShotsOnTargetPercent,
        j."Sh/90" as ShotsPer90,
        j."SoT/90" as ShotsOnTargetPer90,
        j."G/Sh" as GoalsPerShot,
        j."G/SoT" as GoalPerShotOnTarget

        FROM predict_df_temp2 DF 

        LEFT JOIN player_shooting_df j
        ON DF.Season = j.Year
        AND DF.Player = J.Player
        AND DF.Pos = j.Pos
        AND DF.Nation = j.Nation
        AND DF.FbrefClub = j.Squad
        """

predict_df_temp2 = ps.sqldf(q1, locals())

predict_df_temp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13102 entries, 0 to 13101
Columns: 109 entries, Name to GoalPerShotOnTarget
dtypes: float64(31), int64(46), object(32)
memory usage: 10.9+ MB


In [445]:
print(predict_df_temp2.notnull().sum()[-30:])

ClubInvolvedGoalPerShot            9545
ClubInvolvedGoalPerShotOnTarget    9545
ClubInvolvedGoalAgainstPer90       9517
ClubInvolvedShotOnTargetAgainst    9517
ClubInvolvedSaves                  9517
ClubInvolvedSaveRate               9517
ClubInvolvedCleanSheets            9738
ClubInvolvedCleanSheetsRate        9738
MatchesPlayed                      6039
Starts                             6039
MinutesPlayed                      6039
MinutesPlayedPer90                 6039
Goals                              6039
Assists                            6031
NonPenaltyGoals                    6039
YellowCards                        6039
RedCards                           6039
GoalsPer90                         6039
AssistsPer90                       6031
GoalsAssistsPer90                  6031
Player                             6039
Nation                             6037
FbrefClub                          6039
Shots                              5762
ShotsOnTarget                      5762


In [446]:
predict_df_temp2.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/predict_df_temp2.csv")

In [448]:
#join tables with train df
q1 = """SELECT DF.*,
        k.MP as GKMatchPlayed,
        k.Starts as GKStarts,
        k.Min as GKMin,
        k."90s" as "GK90s",
        k.GA as GKGoalAgainst,
        k.GA90 as GKGoalAgainstPer90,
        k.Saves as GKSaves,
        k."Save%" as GKSavePercent,
        k.W as GKWin,
        k.D as GKDraw,
        k.L as GKLost,
        k.CS as GKCleanSheet,
        k."CS%" as GKCleanSheetPercent

        FROM predict_df_temp2 DF 

        LEFT JOIN player_gk_df k
        ON DF.Season = k.Year
        AND DF.ClubInvolved LIKE '%' || k.Squad || '%'
        AND (k.NEWNAME = DF.Name
        OR k.Player = DF.Name
        OR k.LASTNAME = DF.Name
        OR k.FIRSTLASTNAME = DF.Name)
        AND DF.Position = 'GK'
        """

predict_df_temp2 = ps.sqldf(q1, locals())

predict_df_temp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13102 entries, 0 to 13101
Columns: 122 entries, Name to GKCleanSheetPercent
dtypes: float64(31), int64(46), object(45)
memory usage: 12.2+ MB


In [449]:
print(predict_df_temp2.notnull().sum()[-30:])

Assists                 6031
NonPenaltyGoals         6039
YellowCards             6039
RedCards                6039
GoalsPer90              6039
AssistsPer90            6031
GoalsAssistsPer90       6031
Player                  6039
Nation                  6037
FbrefClub               6039
Shots                   5762
ShotsOnTarget           5762
ShotsOnTargetPercent    5311
ShotsPer90              5762
ShotsOnTargetPer90      5762
GoalsPerShot            5311
GoalPerShotOnTarget     4624
GKMatchPlayed            559
GKStarts                 559
GKMin                    559
GK90s                    559
GKGoalAgainst            546
GKGoalAgainstPer90       546
GKSaves                  546
GKSavePercent            543
GKWin                    559
GKDraw                   559
GKLost                   559
GKCleanSheet             559
GKCleanSheetPercent      544
dtype: int64


In [450]:
predict_df_temp2.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/predict_df_temp3.csv")

In [301]:
#join tables with train df
q1 = """SELECT DF.*,
        m.Contract ContractGood

        FROM train_df_temp2 DF 

        LEFT JOIN df_contract m
        ON DF.Season = m.Season + 1
        AND DF.ID = m.ID
        """

train_contract_df = ps.sqldf(q1, locals())

train_contract_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17661 entries, 0 to 17660
Columns: 271 entries, Unnamed: 0 to ContractGood
dtypes: float64(170), int64(35), object(66)
memory usage: 36.5+ MB


In [302]:
train_contract_df.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_df_temporary4.csv")

In [259]:
train_df_ws = train_df_ws.drop('Age', 1)

In [260]:
train_df_ws = train_df_ws.drop('name', 1)

In [71]:
print(list(train_df_ws.columns))

['NewName', 'MarketValue', 'ClubInvolved', 'League', 'MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'ID', 'Name', 'Height_cm', 'Weight_kg', 'Country', 'Club', 'Joined', 'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing', 'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration', 'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power', 'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'GK_diving', 'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes', 'Value', 'Wage', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'Fee', 'Contract', 'ClubInvolvedRank', 'ClubInvolvedWin', 'ClubInvolvedDraw', 'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints', 'ClubRank', 'ClubWi

In [269]:
train_df_temp2.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_df_temp3.xlsx")

In [270]:
train_df_temp2.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_df_temp3.csv")

In [266]:
test = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_df_temp3.csv")
test.head(20)

/Users/roland.kovacs4ibm.com/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (15,65,122,123,124,125,126,127,128,131,132,133,135) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,Unnamed: 0.1,NewName,MarketValue,ClubInvolved,League,MainPosition,LeagueInvolved,CorrMarketValue,ID,...,GoalPerShotOnTarget,GKGoalAgainst,GKGoalAgainstPer90,GKSaves,GKSavePercent,GKWin,GKDraw,GKLost,GKCleanSheet,GKCleanSheetPercent
0,0,0,Bernard,10.00,Clube Atlético Mineiro,Ukrainian Premier League,Forward,Brazil,5.000,205525,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,Fred,5.00,Sport Club Internacional,Ukrainian Premier League,Midfielder,Brazil,2.500,209297,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,Fernando,10.00,Grêmio Foot-Ball Porto Alegrense,Ukrainian Premier League,Midfielder,Brazil,5.000,202642,...,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,W. Nem,8.00,Fluminense Football Club,Ukrainian Premier League,Forward,Brazil,4.000,203279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,F. Ferreyra,6.00,Club Atlético Vélez Sarsfield,Ukrainian Premier League,Forward,Argentina,3.000,213432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,5,O. Volovyk,1.50,Metalurg Donetsk,Ukrainian Premier League,Defender,Ukrainian Premier League,0.750,216427,...,0.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,6,A. Shynder,1.70,SK Tavriya Simferopol,Ukrainian Premier League,Forward,Ukrainian Premier League,0.850,206432,...,0.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,7,P. André Helland,0.15,IL Hödd,Norwegian Eliteserien,Forward,Norway,0.075,183336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,8,J. Skjelvik,0.50,Kalmar FF,Norwegian Eliteserien,Defender,Swedish Allsvenskan,0.250,188917,...,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,9,A. Gundersen,0.00,Pors Grenland,Norwegian Eliteserien,Goalkeeper,Norway,0.000,219901,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:

player_standard_df = player_standard_df.replace('Aras Ãzbiliz', 'A. Özbiliz', regex=True)
player_shooting_df = player_shooting_df.replace('Aras Ãzbiliz', 'A. Özbiliz', regex=True)
player_gk_df = player_gk_df.replace('Aras Ãzbiliz', 'A. Özbiliz', regex=True)


player_standard_df = player_standard_df.replace('BeÅiktaÅ	', 'Besiktas JK', regex=True)
player_shooting_df = player_shooting_df.replace('BeÅiktaÅ	', 'Besiktas JK', regex=True)
player_gk_df = player_gk_df.replace('BeÅiktaÅ	', 'Besiktas JK', regex=True)


player_standard_df = player_standard_df.replace('Ã©', 'é', regex=True)
player_shooting_df = player_shooting_df.replace('Ã©', 'é', regex=True)
player_gk_df = player_gk_df.replace('Ã©', 'é', regex=True)

player_standard_df = player_standard_df.replace('Ãº', 'ú', regex=True)
player_shooting_df = player_shooting_df.replace('Ãº', 'ú', regex=True)
player_gk_df = player_gk_df.replace('Ãº', 'ú', regex=True)


player_standard_df = player_standard_df.replace('Å¡', 's', regex=True)
player_shooting_df = player_shooting_df.replace('Å¡', 's', regex=True)
player_gk_df = player_gk_df.replace('Å¡', 's', regex=True)


player_standard_df = player_standard_df.replace('Ã¼', 'ü', regex=True)
player_shooting_df = player_shooting_df.replace('Ã¼', 'ü', regex=True)
player_gk_df = player_gk_df.replace('Ã¼', 'ü', regex=True)


player_standard_df = player_standard_df.replace('Ã¶', 'ö', regex=True)
player_shooting_df = player_shooting_df.replace('Ã¶', 'ö', regex=True)
player_gk_df = player_gk_df.replace('Ã¶', 'ö', regex=True)


player_standard_df = player_standard_df.replace('Ã§', 'c', regex=True)
player_shooting_df = player_shooting_df.replace('Ã§', 'c', regex=True)
player_gk_df = player_gk_df.replace('Ã§', 'c', regex=True)


player_standard_df = player_standard_df.replace('è', 'é', regex=True)
player_shooting_df = player_shooting_df.replace('è', 'é', regex=True)
player_gk_df = player_gk_df.replace('è', 'é', regex=True)


player_standard_df = player_standard_df.replace('Ã¨', 'é', regex=True)
player_shooting_df = player_shooting_df.replace('Ã¨', 'é', regex=True)
player_gk_df = player_gk_df.replace('Ã¨', 'é', regex=True)


player_standard_df = player_standard_df.replace('ş', 's', regex=True)
player_shooting_df = player_shooting_df.replace('ş', 's', regex=True)
player_gk_df = player_gk_df.replace('ş', 's', regex=True)


player_standard_df = player_standard_df.replace('Ã«', 'e', regex=True)
player_shooting_df = player_shooting_df.replace('Ã«', 'e', regex=True)
player_gk_df = player_gk_df.replace('Ã«', 'e', regex=True)


player_standard_df = player_standard_df.replace('î', 'i', regex=True)
player_shooting_df = player_shooting_df.replace('î', 'i', regex=True)
player_gk_df = player_gk_df.replace('î', 'i', regex=True)


player_standard_df = player_standard_df.replace('Ã®', 'i', regex=True)
player_shooting_df = player_shooting_df.replace('Ã®', 'i', regex=True)
player_gk_df = player_gk_df.replace('Ã®', 'i', regex=True)


player_standard_df = player_standard_df.replace('Ã³', 'ó', regex=True)
player_shooting_df = player_shooting_df.replace('Ã³', 'ó', regex=True)
player_gk_df = player_gk_df.replace('Ã³', 'ó', regex=True)


player_standard_df = player_standard_df.replace('Å¾', 'z', regex=True)
player_shooting_df = player_shooting_df.replace('Å¾', 'z', regex=True)
player_gk_df = player_gk_df.replace('Å¾', 'z', regex=True)


player_standard_df = player_standard_df.replace('Ã±', 'n', regex=True)
player_shooting_df = player_shooting_df.replace('Ã±', 'n', regex=True)
player_gk_df = player_gk_df.replace('Ã±', 'n', regex=True)

player_standard_df = player_standard_df.replace('ä', 'a', regex=True)
player_shooting_df = player_shooting_df.replace('ä', 'a', regex=True)
player_gk_df = player_gk_df.replace('ä', 'a', regex=True)

player_standard_df = player_standard_df.replace('Ã¤', 'a', regex=True)
player_shooting_df = player_shooting_df.replace('Ã¤', 'a', regex=True)
player_gk_df = player_gk_df.replace('Ã¤', 'a', regex=True)

player_standard_df = player_standard_df.replace('ø', 'o', regex=True)
player_shooting_df = player_shooting_df.replace('ø', 'o', regex=True)
player_gk_df = player_gk_df.replace('ø', 'o', regex=True)

player_standard_df = player_standard_df.replace('Ã½', 'y', regex=True)
player_shooting_df = player_shooting_df.replace('Ã½', 'y', regex=True)
player_gk_df = player_gk_df.replace('Ã½', 'y', regex=True)

player_standard_df = player_standard_df.replace('Ã²', 'ó', regex=True)
player_shooting_df = player_shooting_df.replace('Ã²', 'ó', regex=True)
player_gk_df = player_gk_df.replace('Ã²', 'ó', regex=True)

player_standard_df = player_standard_df.replace('Ã¸', 'ö', regex=True)
player_shooting_df = player_shooting_df.replace('Ã¸', 'ö', regex=True)
player_gk_df = player_gk_df.replace('Ã¸', 'ö', regex=True)

player_standard_df = player_standard_df.replace('Ã£', 'á', regex=True)
player_shooting_df = player_shooting_df.replace('Ã£', 'á', regex=True)
player_gk_df = player_gk_df.replace('Ã£', 'á', regex=True)

player_standard_df = player_standard_df.replace('Ã¯', 'i', regex=True)
player_shooting_df = player_shooting_df.replace('Ã¯', 'i', regex=True)
player_gk_df = player_gk_df.replace('Ã¯', 'i', regex=True)

player_standard_df = player_standard_df.replace('Ã´', 'o', regex=True)
player_shooting_df = player_shooting_df.replace('Ã´', 'o', regex=True)
player_gk_df = player_gk_df.replace('Ã´', 'o', regex=True)


player_standard_df = player_standard_df.replace('Å½', 'Z', regex=True)
player_shooting_df = player_shooting_df.replace('Å½', 'Z', regex=True)
player_gk_df = player_gk_df.replace('Å½', 'Z', regex=True)

player_standard_df = player_standard_df.replace('Ã§', 'c', regex=True)
player_shooting_df = player_shooting_df.replace('Ã§', 'c', regex=True)
player_gk_df = player_gk_df.replace('Ã§', 'c', regex=True)


player_standard_df = player_standard_df.replace('Å¯', 'u', regex=True)
player_shooting_df = player_shooting_df.replace('Å¯', 'u', regex=True)
player_gk_df = player_gk_df.replace('Å¯', 'u', regex=True)

player_standard_df = player_standard_df.replace('Ä°', 'I', regex=True)
player_shooting_df = player_shooting_df.replace('Ä°', 'I', regex=True)
player_gk_df = player_gk_df.replace('Ä°', 'I', regex=True)

player_standard_df = player_standard_df.replace('Ä½', 'L', regex=True)
player_shooting_df = player_shooting_df.replace('Ä½', 'L', regex=True)
player_gk_df = player_gk_df.replace('Ä½', 'L', regex=True)

player_standard_df = player_standard_df.replace('Ä¾', 'l', regex=True)
player_shooting_df = player_shooting_df.replace('Ä¾', 'l', regex=True)
player_gk_df = player_gk_df.replace('Ä¾', 'l', regex=True)

player_standard_df = player_standard_df.replace('Ã¶', 'ö', regex=True)
player_shooting_df = player_shooting_df.replace('Ã¶', 'ö', regex=True)
player_gk_df = player_gk_df.replace('Ã¶', 'ö', regex=True)

player_standard_df = player_standard_df.replace('Å¼', 'z', regex=True)
player_shooting_df = player_shooting_df.replace('Å¼', 'z', regex=True)
player_gk_df = player_gk_df.replace('Å¼', 'z', regex=True)

player_standard_df = player_standard_df.replace('Ä±', 'i', regex=True)
player_shooting_df = player_shooting_df.replace('Ä±', 'i', regex=True)
player_gk_df = player_gk_df.replace('Ä±', 'i', regex=True)

player_standard_df = player_standard_df.replace('Ã¡', 'á', regex=True)
player_shooting_df = player_shooting_df.replace('Ã¡', 'á', regex=True)
player_gk_df = player_gk_df.replace('Ã¡', 'á', regex=True)

player_standard_df = player_standard_df.replace('Ã¥', 'a', regex=True)
player_shooting_df = player_shooting_df.replace('Ã¥', 'a', regex=True)
player_gk_df = player_gk_df.replace('Ã¥', 'a', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'c', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'c', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'c', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'C', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'C', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'C', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'a', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'a', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'a', regex=True)

player_standard_df = player_standard_df.replace('Å', 's', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 's', regex=True)
player_gk_df = player_gk_df.replace('Å', 's', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'g', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'g', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'g', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'C', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'C', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'C', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'c', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'c', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'c', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'D', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'D', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'D', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'd', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'd', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'd', regex=True)

player_standard_df = player_standard_df.replace('Ä¢', 'G', regex=True)
player_shooting_df = player_shooting_df.replace('Ä¢', 'G', regex=True)
player_gk_df = player_gk_df.replace('Ä¢', 'G', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'e', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'e', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'e', regex=True)

player_standard_df = player_standard_df.replace('Å', 'n', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'n', regex=True)
player_gk_df = player_gk_df.replace('Å', 'n', regex=True)

player_standard_df = player_standard_df.replace('Å', 'l', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'l', regex=True)
player_gk_df = player_gk_df.replace('Å', 'l', regex=True)

player_standard_df = player_standard_df.replace('Å', 'S', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'S', regex=True)
player_gk_df = player_gk_df.replace('Å', 'S', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'a', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'a', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'a', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'e', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'e', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'e', regex=True)

player_standard_df = player_standard_df.replace('Å', 'L', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'L', regex=True)
player_gk_df = player_gk_df.replace('Å', 'L', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'e', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'e', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'e', regex=True)

player_standard_df = player_standard_df.replace('Åº', 'z', regex=True)
player_shooting_df = player_shooting_df.replace('Åº', 'z', regex=True)
player_gk_df = player_gk_df.replace('Åº', 'z', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'a', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'a', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'a', regex=True)

player_standard_df = player_standard_df.replace('Å ', 'S', regex=True)
player_shooting_df = player_shooting_df.replace('Å ', 'S', regex=True)
player_gk_df = player_gk_df.replace('Å ', 'S', regex=True)

player_standard_df = player_standard_df.replace('Å', 'n', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'n', regex=True)
player_gk_df = player_gk_df.replace('Å', 'n', regex=True)

player_standard_df = player_standard_df.replace('Ä¼', 'l', regex=True)
player_shooting_df = player_shooting_df.replace('Ä¼', 'l', regex=True)
player_gk_df = player_gk_df.replace('Ä¼', 'l', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'é', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'é', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'é', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'D', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'D', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'D', regex=True)

player_standard_df = player_standard_df.replace('Å', 'r', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'r', regex=True)
player_gk_df = player_gk_df.replace('Å', 'r', regex=True)

player_standard_df = player_standard_df.replace('Å ', 'S', regex=True)
player_shooting_df = player_shooting_df.replace('Å ', 'S', regex=True)
player_gk_df = player_gk_df.replace('Å ', 'S', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'e', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'e', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'e', regex=True) 

player_standard_df = player_standard_df.replace('Ä«', 'i', regex=True)
player_shooting_df = player_shooting_df.replace('Ä«', 'i', regex=True)
player_gk_df = player_gk_df.replace('Ä«', 'i', regex=True)

player_standard_df = player_standard_df.replace('Ä·', 'k', regex=True)
player_shooting_df = player_shooting_df.replace('Ä·', 'k', regex=True)
player_gk_df = player_gk_df.replace('Ä·', 'k', regex=True)

player_standard_df = player_standard_df.replace('Ä', 'e', regex=True)
player_shooting_df = player_shooting_df.replace('Ä', 'e', regex=True)
player_gk_df = player_gk_df.replace('Ä', 'e', regex=True)

player_standard_df = player_standard_df.replace('Å', 'S', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'S', regex=True)
player_gk_df = player_gk_df.replace('Å', 'S', regex=True)

player_standard_df = player_standard_df.replace('Å«', 'u', regex=True)
player_shooting_df = player_shooting_df.replace('Å«', 'u', regex=True)
player_gk_df = player_gk_df.replace('Å«', 'u', regex=True)

player_standard_df = player_standard_df.replace('Å', 'O', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'O', regex=True)
player_gk_df = player_gk_df.replace('Å', 'O', regex=True)

player_standard_df = player_standard_df.replace('Å', 's', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 's', regex=True)
player_gk_df = player_gk_df.replace('Å', 's', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'Á', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'Á', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'Á', regex=True)

player_standard_df = player_standard_df.replace('Ã°', 'd', regex=True)
player_shooting_df = player_shooting_df.replace('Ã°', 'd', regex=True)
player_gk_df = player_gk_df.replace('Ã°', 'd', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'Á', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'Á', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'Á', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'ß', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'ß', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'ß', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'E', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'E', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'E', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'Ö', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'Ö', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'Ö', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'Ó', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'Ó', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'Ó', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'O', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'O', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'O', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'Ü', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'Ü', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'Ü', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'C', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'C', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'C', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'Th', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'Th', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'Th', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'C', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'C', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'C', regex=True)

player_standard_df = player_standard_df.replace('Ã¾', 'th', regex=True)
player_shooting_df = player_shooting_df.replace('Ã¾', 'th', regex=True)
player_gk_df = player_gk_df.replace('Ã¾', 'th', regex=True)

player_standard_df = player_standard_df.replace('Ã¢', 'a', regex=True)
player_shooting_df = player_shooting_df.replace('Ã¢', 'a', regex=True)
player_gk_df = player_gk_df.replace('Ã¢', 'a', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'Í', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'Í', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'Í', regex=True)

player_standard_df = player_standard_df.replace('Ã', 'í', regex=True)
player_shooting_df = player_shooting_df.replace('Ã', 'í', regex=True)
player_gk_df = player_gk_df.replace('Ã', 'í', regex=True)

player_standard_df = player_standard_df.replace('Å', 'R', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'R', regex=True)
player_gk_df = player_gk_df.replace('Å', 'R', regex=True)

player_standard_df = player_standard_df.replace('Å', 'o', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'o', regex=True)
player_gk_df = player_gk_df.replace('Å', 'o', regex=True)

player_standard_df = player_standard_df.replace('Å', 'o', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'o', regex=True)
player_gk_df = player_gk_df.replace('Å', 'o', regex=True)

player_standard_df = player_standard_df.replace('Å»', 'Z', regex=True)
player_shooting_df = player_shooting_df.replace('Å»', 'Z', regex=True)
player_gk_df = player_gk_df.replace('Å»', 'Z', regex=True)

player_standard_df = player_standard_df.replace('Å¥', 't', regex=True)
player_shooting_df = player_shooting_df.replace('Å¥', 't', regex=True)
player_gk_df = player_gk_df.replace('Å¥', 't', regex=True)

player_standard_df = player_standard_df.replace('Å', 'n', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'n', regex=True)
player_gk_df = player_gk_df.replace('Å', 'n', regex=True)

player_standard_df = player_standard_df.replace('Å', 'S', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'S', regex=True)
player_gk_df = player_gk_df.replace('Å', 'S', regex=True)

In [254]:
player_standard_df = player_standard_df.replace('Å', 'S', regex=True)
player_shooting_df = player_shooting_df.replace('Å', 'S', regex=True)
player_gk_df = player_gk_df.replace('Å', 'S', regex=True)

In [252]:
player_shooting_df[player_shooting_df['Player'].str.contains("Å") == True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 922 entries, 2446 to 233994
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     922 non-null    int64  
 1   Unnamed: 0.1   831 non-null    float64
 2   Rk             922 non-null    object 
 3   Player         922 non-null    object 
 4   Nation         915 non-null    object 
 5   Pos            921 non-null    object 
 6   Squad          922 non-null    object 
 7   Age            921 non-null    object 
 8   Born           921 non-null    object 
 9   90s            922 non-null    object 
 10  Gls            922 non-null    object 
 11  Sh             148 non-null    object 
 12  SoT            782 non-null    object 
 13  SoT%           45 non-null     object 
 14  Sh/90          148 non-null    object 
 15  SoT/90         782 non-null    object 
 16  G/Sh           45 non-null     object 
 17  G/SoT          561 non-null    object 
 18  Dist

In [255]:
player_shooting_df[player_shooting_df['Player'].str.contains("Ã") == True].tail(50)

,Unnamed: 0,Unnamed: 0.1,Rk,Player,Nation,Pos,Squad,Age,Born,90s,...,G/SoT,Dist,PK,PKatt,Season,League,Year,NewName,LastName,FirstLastName


In [256]:
player_standard_df.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/player_standard_df.csv")
player_shooting_df.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/player_shooting_df.csv")
player_gk_df.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/player_gk_df.csv")
player_standard_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/player_standard_df.xlsx")
player_shooting_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/player_shooting_df.xlsx")
player_gk_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/player_gk_df.xlsx")

In [2]:
df = pd.read_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_df_temp.xlsx")

Whoscored

In [451]:
who_scored = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/WhoScoredFinal1_filtered.cs", sep=',')
who_scored['lastName'].head()

/Users/roland.kovacs4ibm.com/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


0                Lewandowski
1                      Messi
2                      Kroos
3                       Kane
4    Alcántara do Nascimento
Name: lastName, dtype: object

In [4]:
who_scored.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142723 entries, 0 to 142722
Columns: 115 entries, seasonName to assist
dtypes: float64(98), int64(6), object(11)
memory usage: 125.2+ MB


In [452]:
who_scored = who_scored.drop_duplicates()

In [453]:
who_scored['Player'] = who_scored['firstName'] + ' ' +  who_scored['lastName']

In [454]:
who_scored['NewName'] = None
who_scored['SecondName'] = None
who_scored['FirstLastName'] = None
for i, r in who_scored.iterrows():
    name = who_scored.at[i,'name']
    try:
        if ' ' in name:
            str_pos = name.find(' ')
            first_char = name[0]
            new_name = first_char + '. ' + name[str_pos+1:]
            who_scored.at[i,'NewName'] = new_name

            end_pos = name.rfind(' ')
            who_scored.at[i,'SecondName'] = name[end_pos+1:]

            who_scored.at[i,'FirstLastName'] = first_char + '. ' + name[end_pos+1:]
        else:
            who_scored.at[i,'NewName'] = who_scored.at[i,'name']
            who_scored.at[i,'SecondName'] = who_scored.at[i,'name']
            who_scored.at[i,'FirstLastName'] = who_scored.at[i,'name']
    except:
        pass

In [455]:
who_scored_a = who_scored[who_scored.goal_x.isnull()==False][['seasonName', 'tournamentRegionName', 'tournamentName', 'League', 'firstName', 'lastName', 'NewName', 'SecondName',
'FirstLastName', 'playerId', 'teamId',	'teamName',	'teamRegionName', 'playedPositions', 'age', 'positionText',
'apps', 'subOn', 'minsPlayed', 'rating', 'goal_x', 'assistTotal_x', 'yellowCard_x',	'redCard_x', 'shotsPerGame_x', 'aerialWonPerGame', 'manOfTheMatch', 'name',
'passSuccess_x', 'tacklePerGame', 'interceptionPerGame', 'foulsPerGame', 'offsideWonPerGame', 'clearancePerGame', 'wasDribbledPerGame', 'outfielderBlockPerGame',
'goalOwn_x', 'keyPassPerGame_x', 'dribbleWonPerGame', 'foulGivenPerGame', 'offsideGivenPerGame', 'dispossessedPerGame', 'turnoverPerGame',
'totalPassesPerGame', 'accurateCrossesPerGame', 'accurateLongPassPerGame', 'accurateThroughBallPerGame']]

In [456]:
who_scored_a = who_scored_a.drop_duplicates()

In [457]:
who_scored_a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52687 entries, 0 to 142231
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   seasonName                  52687 non-null  object 
 1   tournamentRegionName        52687 non-null  object 
 2   tournamentName              52687 non-null  object 
 3   League                      51023 non-null  object 
 4   firstName                   52687 non-null  object 
 5   lastName                    52687 non-null  object 
 6   NewName                     52687 non-null  object 
 7   SecondName                  52687 non-null  object 
 8   FirstLastName               52687 non-null  object 
 9   playerId                    52687 non-null  int64  
 10  teamId                      52687 non-null  int64  
 11  teamName                    52687 non-null  object 
 12  teamRegionName              52687 non-null  object 
 13  playedPositions             51

In [458]:
who_scored_b = who_scored[who_scored.tackleTotalAttempted.isnull()==False][['seasonName', 'tournamentRegionName', 'tournamentName', 'firstName', 'lastName', 'playerId',
'NewName', 'SecondName', 'FirstLastName', 'teamId', 'teamName',	'teamRegionName', 'playedPositions', 'age',	'positionText',
'tackleTotalAttempted',	'challengeLost', 'interceptionAll', 'foulCommitted', 'passCrossBlockedDefensive', 'outfielderBlockedPass', 'saveSixYardBox', 'savePenaltyArea',
'saveObox',	'saveTotal', 'shotSixYardBox', 'shotPenaltyArea', 'shotOboxTotal', 'shotsTotal_x', 'shotOpenPlay', 'shotCounter', 'shotSetPiece', 'penaltyTaken',
'shotOnTarget',	'shotOffTarget', 'shotOnPost', 'shotBlocked', 'shotRightFoot', 'shotLeftFoot', 'shotHead', 'goalSixYardBox', 'goalPenaltyArea',	'goalObox',
'goalTotal_x', 'goalOwn_y',	'goalOpenPlay',	'goalCounter', 'goalSetPiece', 'penaltyScored', 'goalRightFoot', 'goalLeftFoot', 'goalHead', 'dribbleLost', 'dribbleTotal',
'duelAerialLost', 'duelAerialTotal', 'duelAerialWon', 'passTotal', 'passLongBallAccurate', 'passLongBallInaccurate', 'shortPassAccurate', 'shortPassInaccurate',
'passCornerAccurate', 'passCornerInaccurate', 'passFreekickAccurate', 'passFreekickInaccurate',	'passCrossInaccurate', 'passCrossAccurate',	'keyPassLong', 'keyPassShort',
'keyPassesTotal', 'keyPassCross', 'keyPassCorner', 'keyPassThroughball', 'keyPassFreekick',	'keyPassThrowin', 'keyPassOther', 'assistCross', 'assistCorner', 'assistThroughball',
'assistFreekick', 'assistThrowin', 'assistOther', 'assist']]

In [459]:
who_scored_b = who_scored_b.drop_duplicates()

In [460]:
who_scored_b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90268 entries, 0 to 142722
Data columns (total 84 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   seasonName                 90268 non-null  object 
 1   tournamentRegionName       90268 non-null  object 
 2   tournamentName             90268 non-null  object 
 3   firstName                  90260 non-null  object 
 4   lastName                   90260 non-null  object 
 5   playerId                   90268 non-null  int64  
 6   NewName                    90268 non-null  object 
 7   SecondName                 90268 non-null  object 
 8   FirstLastName              90268 non-null  object 
 9   teamId                     90268 non-null  int64  
 10  teamName                   90268 non-null  object 
 11  teamRegionName             90268 non-null  object 
 12  playedPositions            81238 non-null  object 
 13  age                        90268 non-null  in

In [461]:
#join tables with train df
q1 = """SELECT DF.*,
        i.tackleTotalAttempted, i.challengeLost, i.interceptionAll, i.foulCommitted, i.passCrossBlockedDefensive, i.outfielderBlockedPass, i.saveSixYardBox, i.savePenaltyArea,
        i.saveObox, i.saveTotal, i.shotSixYardBox, i.shotPenaltyArea, i.shotOboxTotal, i.shotsTotal_x, i.shotOpenPlay, i.shotCounter, i.shotSetPiece, i.penaltyTaken,
        i.shotOnTarget,	i.shotOffTarget, i.shotOnPost, i.shotBlocked, i.shotRightFoot, i.shotLeftFoot, i.shotHead, i.goalSixYardBox, i.goalPenaltyArea,	i.goalObox,
        i.goalTotal_x, i.goalOwn_y, i.goalOpenPlay, i.goalCounter, i.goalSetPiece, i.penaltyScored, i.goalRightFoot, i.goalLeftFoot, i.goalHead, i.dribbleLost, i.dribbleTotal,
        i.duelAerialLost, i.duelAerialTotal, i.duelAerialWon, i.passTotal, i.passLongBallAccurate, i.passLongBallInaccurate, i.shortPassAccurate, i.shortPassInaccurate,
        i.passCornerAccurate, i.passCornerInaccurate, i.passFreekickAccurate, i.passFreekickInaccurate,	i.passCrossInaccurate, i.passCrossAccurate, i.keyPassLong, i.keyPassShort,
        i.keyPassesTotal, i.keyPassCross, i.keyPassCorner, i.keyPassThroughball, i.keyPassFreekick, i.keyPassThrowin, i.keyPassOther, i.assistCross, i.assistCorner, i.assistThroughball,
        i.assistFreekick, i.assistThrowin, i.assistOther, i.assist
      

        FROM who_scored_a DF 

        LEFT JOIN who_scored_b i
        ON DF.seasonName = i.seasonName
        AND DF.tournamentName = i.tournamentName
        AND i.firstName = DF.firstName
        AND i.lastName = DF.lastName
        AND i.playerId = DF.playerId
        AND i.teamId = DF.teamId


        """

ws_clean = ps.sqldf(q1, locals())

ws_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65793 entries, 0 to 65792
Columns: 116 entries, seasonName to assist
dtypes: float64(96), int64(6), object(14)
memory usage: 58.2+ MB


In [462]:
ws_df = ws_clean[(ws_clean.tournamentName.isin(['Champions League', 'Europa League']))==False]

In [463]:
ws_df[ws_df.NewName == 'E. Hazard']

,seasonName,tournamentRegionName,tournamentName,League,firstName,lastName,NewName,SecondName,FirstLastName,playerId,...,keyPassFreekick,keyPassThrowin,keyPassOther,assistCross,assistCorner,assistThroughball,assistFreekick,assistThrowin,assistOther,assist
27204,2018/2019,England,Premier League,English Premier League,Eden,Hazard,E. Hazard,Hazard,E. Hazard,33404,...,0.162162,0.0,1.702703,0.108108,0.054054,0.027027,0.027027,0.0,0.270270,0.405405
27488,2017/2018,England,Premier League,English Premier League,Eden,Hazard,E. Hazard,Hazard,E. Hazard,33404,...,0.029412,0.0,2.176471,0.000000,0.000000,0.029412,0.000000,0.0,0.088235,0.117647
27775,2016/2017,England,Premier League,English Premier League,Eden,Hazard,E. Hazard,Hazard,E. Hazard,33404,...,0.000000,0.0,2.222222,0.000000,0.000000,0.000000,0.000000,0.0,0.138889,0.138889
28121,2015/2016,England,Premier League,English Premier League,Eden,Hazard,E. Hazard,Hazard,E. Hazard,33404,...,0.000000,0.0,1.903226,0.000000,0.000000,0.000000,0.000000,0.0,0.096774,0.096774
28357,2014/2015,England,Premier League,English Premier League,Eden,Hazard,E. Hazard,Hazard,E. Hazard,33404,...,0.000000,0.0,2.394737,0.026316,0.000000,0.000000,0.000000,0.0,0.210526,0.236842
28657,2013/2014,England,Premier League,English Premier League,Eden,Hazard,E. Hazard,Hazard,E. Hazard,33404,...,0.000000,0.0,2.342857,0.028571,0.000000,0.085714,0.000000,0.0,0.085714,0.200000
28963,2012/2013,England,Premier League,English Premier League,Eden,Hazard,E. Hazard,Hazard,E. Hazard,33404,...,0.029412,0.0,1.588235,0.029412,0.000000,0.058824,0.000000,0.0,0.235294,0.323529
41854,2011/2012,France,Ligue 1,French Ligue 1,Eden,Hazard,E. Hazard,Hazard,E. Hazard,33404,...,0.263158,0.0,1.578947,0.236842,0.052632,0.078947,0.052632,0.0,0.105263,0.421053
42145,2010/2011,France,Ligue 1,French Ligue 1,Eden,Hazard,E. Hazard,Hazard,E. Hazard,33404,...,0.157895,0.0,2.000000,0.052632,0.000000,0.026316,0.000000,0.0,0.184211,0.263158
42464,2009/2010,France,Ligue 1,French Ligue 1,Eden,Hazard,E. Hazard,Hazard,E. Hazard,33404,...,0.108108,0.0,0.972973,0.081081,0.000000,0.054054,0.027027,0.0,0.081081,0.216216


In [472]:
set(ws_df.seasonName.values)

{' ',
 '2009/2010',
 '2010/2011',
 '2011/2012',
 '2012/2013',
 '2013',
 '2013/2014',
 '2014',
 '2014/2015',
 '2015',
 '2015/2016',
 '2016',
 '2016/2017',
 '2017',
 '2017/2018',
 '2018',
 '2018/2019',
 '2019',
 '2019/2020',
 '2020'}

In [464]:
def seasonName_add(row):
   if row['seasonName'] == '2020/2021' :
      return '2022'
   if row['seasonName'] == '2019/2020' :
      return '2021'
   if row['seasonName'] == '2018/2019' :
      return '2020'
   if row['seasonName'] == '2017/2018':
      return '2019'
   if row['seasonName'] == '2016/2017':
      return '2018'
   if row['seasonName'] == '2015/2016':
      return '2017'
   if row['seasonName'] == '2014/2015':
      return '2016'
   if row['seasonName'] == '2013/2014':
      return '2015'
   if row['seasonName'] == '2012/2013':
      return '2014'
   if row['seasonName'] == '2011/2012':
      return '2013'
   if row['seasonName'] == '2010/2011':
      return '2012'
   if row['seasonName'] == '2009/2010':
      return '2011'
   if row['seasonName'] == '2008/2009':
      return '2010'
   if row['seasonName'] == '2007/2008':
      return '2009'
   if row['seasonName'] == '2006/2007':
      return '2008'

In [465]:
ws_df['Year'] = ws_df.apply(lambda row:seasonName_add(row), axis=1)

/var/folders/8m/8q13t9cn0917nvwcf017zkbc0000gn/T/ipykernel_96711/3345819110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ws_df['Year'] = ws_df.apply(lambda row:seasonName_add(row), axis=1)


In [466]:
ws_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38858 entries, 26935 to 65792
Columns: 117 entries, seasonName to Year
dtypes: float64(96), int64(6), object(15)
memory usage: 35.0+ MB


In [54]:
ws_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/ws_df.xlsx")

In [29]:
ws_df = pd.read_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/ws_df.xlsx")

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17245 entries, 0 to 17244
Columns: 123 entries, Unnamed: 0 to Squad.1
dtypes: float64(68), int64(37), object(18)
memory usage: 16.2+ MB


In [116]:
print(list(df.columns))

['Unnamed: 0', 'Unnamed: 0.1', 'NewName', 'MarketValue', 'ClubInvolved', 'League', 'MainPosition', 'LeagueInvolved', 'CorrMarketValue', 'ID', 'Name', 'Age', 'Height_cm', 'Weight_kg', 'Country', 'Club', 'Joined', 'Position', 'Foot', 'IntReputation', 'Crossing', 'Finishing', 'Heading_accuracy', 'Short_passing', 'Volleys', 'Dribbling', 'Curve', 'FK_accuracy', 'Long_passing', 'Ball_control', 'Acceleration', 'Sprint_speed', 'Agility', 'Reactions', 'Balance', 'Shot_power', 'Jumping', 'Stamina', 'Strength', 'Long_shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'Standing_tackle', 'Sliding_tackle', 'GK_diving', 'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes', 'Value', 'Wage', 'Season', 'Years', 'DEGREE', 'Eigv', 'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA', 'Fee', 'Contract', 'ClubInvolvedRank', 'ClubInvolvedWin', 'ClubInvolvedDraw', 'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'Clu

In [4]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1'], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17245 entries, 0 to 17244
Columns: 121 entries, NewName to Squad.1
dtypes: float64(68), int64(35), object(18)
memory usage: 15.9+ MB


In [467]:
ws_df = ws_df.replace('RasenBallsport Leipzig', 'RB Leipzig', regex=True)
ws_df = ws_df.replace('Bayer Leverkusen', 'Bayer 04 Leverkusen', regex=True)
ws_df = ws_df.replace('Atletico Madrid', 'Atlético Madrid', regex=True)
ws_df = ws_df.replace('Saint-Etienne', 'AS Saint-Étienne', regex=True)
ws_df = ws_df.replace('Thonon Evian', 'Évian Thonon Gaillard FC', regex=True)
ws_df = ws_df.replace('SC Bastia', 'Sporting Club de Bastia', regex=True)
ws_df = ws_df.replace('Rennes', 'Stade Rennais FC', regex=True)
ws_df = ws_df.replace('FC Koln', '1. FC Köln', regex=True)
ws_df = ws_df.replace('Nuernberg', '1. FC Nürnberg', regex=True)
ws_df = ws_df.replace('Bayern Munich', 'FC Bayern München', regex=True)
ws_df = ws_df.replace('Fortuna Duesseldorf', 'Fortuna Düsseldorf', regex=True)
ws_df = ws_df.replace('Hertha Berlin', 'Hertha BSC', regex=True)
ws_df = ws_df.replace('Greuther Fuerth', 'SpVgg Greuther Fürth', regex=True)
ws_df = ws_df.replace('AC Milan', 'Milan', regex=True)
ws_df = ws_df.replace('Parma Calcio 1913', 'Parma', regex=True)
ws_df = ws_df.replace('Athletic Bilbao', 'Athletic Club de Bilbao', regex=True)
ws_df = ws_df.replace('Leganes', 'CD Leganés', regex=True)
ws_df = ws_df.replace('Cordoba', 'Córdoba CF', regex=True)
ws_df = ws_df.replace('Deportivo Alaves', 'Deportivo Alavés', regex=True)
ws_df = ws_df.replace('Deportivo La Coruna', 'Deportivo de La Coruna', regex=True)
ws_df = ws_df.replace('Hercules', 'Hércules CF', regex=True)
ws_df = ws_df.replace('Celta Vigo', 'RC Celta', regex=True)
ws_df = ws_df.replace('Sporting Gijon', 'Real Sporting de Gijón', regex=True)
ws_df = ws_df.replace('Almeria', 'UD Almería', regex=True)
ws_df = ws_df.replace('Arsenal Sarandi', 'Arsenal Fútbol Club', regex=True)
ws_df = ws_df.replace('Atletico Rafaela', 'Atlético de Rafaela', regex=True)
ws_df = ws_df.replace('Colon', 'Club Atlético Colón', regex=True)
ws_df = ws_df.replace('Huracan', 'Club Atlético Huracán', regex=True)
ws_df = ws_df.replace('Lanus', 'Club Atlético Lanús', regex=True)
ws_df = ws_df.replace('San Martin San Juan', 'Club Atlético San Martín (SJ)', regex=True)
ws_df = ws_df.replace('Union', 'Club Atlético Unión', regex=True)
ws_df = ws_df.replace('Velez Sarsfield', 'Club Atlético Vélez Sarsfield', regex=True)
ws_df = ws_df.replace('Gimnasia LP', 'Club de Gimnasia y Esgrima La Plata', regex=True)
ws_df = ws_df.replace('Union', 'Club Atlético Unión', regex=True)
ws_df = ws_df.replace('Sao Paulo', 'São Paulo Futebol Clube', regex=True)
ws_df = ws_df.replace('Athletico Paranaense', 'Atlético Paranaense', regex=True)
ws_df = ws_df.replace('CS Alagoano', 'Centro Sportivo Alagoano (AL)', regex=True)
ws_df = ws_df.replace('Botafogo RJ', 'Botafogo de Futebol e Regatas', regex=True)
ws_df = ws_df.replace('Goias', 'Goiás Esporte Clube', regex=True)
ws_df = ws_df.replace('Associação Chapecoense de Futebol', 'Chapecoense AF', regex=True)
ws_df = ws_df.replace('Ceara', 'Ceará Sporting Club', regex=True)
ws_df = ws_df.replace('Avai', 'Avaí Futebol Clube (SC)', regex=True)
ws_df = ws_df.replace('Sport Club do Recife', 'Sport Recife', regex=True)
ws_df = ws_df.replace('America MG', 'América Futebol Clube (MG)', regex=True)
ws_df = ws_df.replace('Criciuma', 'Criciúma Esporte Clube', regex=True)
ws_df = ws_df.replace('Nautico', 'Clube Náutico Capibaribe', regex=True)
ws_df = ws_df.replace('1860 Muenchen', 'TSV 1860 Munich', regex=True)
ws_df = ws_df.replace('FC Koln', '1. FC Köln', regex=True)
ws_df = ws_df.replace('Fortuna Duesseldorf', 'Fortuna Düsseldorf', regex=True)
ws_df = ws_df.replace('Greuther Fuerth', 'SpVgg Greuther Fürth', regex=True)
ws_df = ws_df.replace('Nuernberg', '1. FC Nürnberg', regex=True)
ws_df = ws_df.replace('Osnabrueck', 'VfL Osnabrück', regex=True)
ws_df = ws_df.replace('Wuerzburger Kickers', 'Würzburger Kickers', regex=True)
ws_df = ws_df.replace('Belenenses SAD', 'Os Belenenses', regex=True)
ws_df = ws_df.replace('Famalicao', 'FC Famalicão', regex=True)
ws_df = ws_df.replace('Maritimo', 'CS Marítimo', regex=True)
ws_df = ws_df.replace('Aves', 'Desportivo Aves', regex=True)
ws_df = ws_df.replace('Vitoria de Guimaraes', 'Vitória Guimarães SC', regex=True)
ws_df = ws_df.replace('Vitoria de Setubal', 'Vitória Setúbal FC', regex=True)
ws_df = ws_df.replace('AFC Eskilstuna', 'Athletic Eskilstuna', regex=True)
ws_df = ws_df.replace('Djurgaarden', 'Djurgardens IF', regex=True)
ws_df = ws_df.replace('IFK Gothenburg', 'IFK Göteborg', regex=True)
ws_df = ws_df.replace('IFK Norrkoeping', 'IFK Norrköping', regex=True)
ws_df = ws_df.replace('Joenkoepings Soedra', 'Jönköpings Södra IF', regex=True)
ws_df = ws_df.replace('Malmoe FF', 'Malmö FF', regex=True)
ws_df = ws_df.replace('Pacos de Ferreira', 'FC Paços de Ferreira', regex=True)
ws_df = ws_df.replace('Oerebro', 'Örebro SK', regex=True)
ws_df = ws_df.replace('Oestersunds FK', 'Östersunds FK', regex=True)
ws_df = ws_df.replace('Ankaragucu', 'MKE Ankaragücü', regex=True)
ws_df = ws_df.replace('Besiktas', 'Beşiktaş JK', regex=True)
ws_df = ws_df.replace('Goztepe', 'Göztepe', regex=True)

In [471]:
ws_df.Year

26935    2021
26936    2021
26937    2021
26938    2021
26939    2021
         ... 
65788    2017
65789    2017
65790    2017
65791    2017
65792    2017
Name: Year, Length: 38858, dtype: object

In [469]:
#join tables with train df
q1 = """SELECT DF.*,
        i.seasonName, i.tournamentRegionName, i.tournamentName, i.League, i.firstName, i.lastName,
        i.FirstLastName, i.playerId, i.teamId, i.teamName, i.teamRegionName, i.playedPositions, i.age, i.positionText,
        i.apps, i.subOn, i.minsPlayed, i.rating, i.goal_x, i.assistTotal_x, i.yellowCard_x, i.redCard_x, i.shotsPerGame_x, i.aerialWonPerGame, i.manOfTheMatch, i.name,
        i.passSuccess_x, i.tacklePerGame, i.interceptionPerGame, i.foulsPerGame, i.offsideWonPerGame, i.clearancePerGame, i.wasDribbledPerGame, i.outfielderBlockPerGame,
        i.goalOwn_x, i.keyPassPerGame_x, i.dribbleWonPerGame, i.foulGivenPerGame, i.offsideGivenPerGame, i.dispossessedPerGame, i.turnoverPerGame,
        i.totalPassesPerGame, i.accurateCrossesPerGame, i.accurateLongPassPerGame, i.accurateThroughBallPerGame,
        i.tackleTotalAttempted, i.challengeLost, i.interceptionAll, i.foulCommitted, i.passCrossBlockedDefensive, i.outfielderBlockedPass, i.saveSixYardBox, i.savePenaltyArea,
        i.saveObox, i.saveTotal, i.shotSixYardBox, i.shotPenaltyArea, i.shotOboxTotal, i.shotsTotal_x, i.shotOpenPlay, i.shotCounter, i.shotSetPiece, i.penaltyTaken,
        i.shotOnTarget,	i.shotOffTarget, i.shotOnPost, i.shotBlocked, i.shotRightFoot, i.shotLeftFoot, i.shotHead, i.goalSixYardBox, i.goalPenaltyArea,	i.goalObox,
        i.goalTotal_x, i.goalOwn_y, i.goalOpenPlay, i.goalCounter, i.goalSetPiece, i.penaltyScored, i.goalRightFoot, i.goalLeftFoot, i.goalHead, i.dribbleLost, i.dribbleTotal,
        i.duelAerialLost, i.duelAerialTotal, i.duelAerialWon, i.passTotal, i.passLongBallAccurate, i.passLongBallInaccurate, i.shortPassAccurate, i.shortPassInaccurate,
        i.passCornerAccurate, i.passCornerInaccurate, i.passFreekickAccurate, i.passFreekickInaccurate,	i.passCrossInaccurate, i.passCrossAccurate, i.keyPassLong, i.keyPassShort,
        i.keyPassesTotal, i.keyPassCross, i.keyPassCorner, i.keyPassThroughball, i.keyPassFreekick, i.keyPassThrowin, i.keyPassOther, i.assistCross, i.assistCorner, i.assistThroughball,
        i.assistFreekick, i.assistThrowin, i.assistOther, i.assist

        FROM predict_df_temp2 DF 

        LEFT JOIN ws_df i
        ON DF.Season = i.Year
        AND (DF.ClubInvolved LIKE '%' || i.teamName || '%')
        AND (i.NewName = DF.Name
        OR i.SecondName = DF.Name
        OR i.FIRSTLASTNAME = DF.Name)
        """

predict_df_ws = ps.sqldf(q1, locals())

predict_df_ws.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13102 entries, 0 to 13101
Columns: 236 entries, Name to assist
dtypes: float64(31), int64(46), object(159)
memory usage: 23.6+ MB


In [470]:
print(predict_df_ws.notnull().sum()[-50:])

shotOffTarget             0
shotOnPost                0
shotBlocked               0
shotRightFoot             0
shotLeftFoot              0
shotHead                  0
goalSixYardBox            0
goalPenaltyArea           0
goalObox                  0
goalTotal_x               0
goalOwn_y                 0
goalOpenPlay              0
goalCounter               0
goalSetPiece              0
penaltyScored             0
goalRightFoot             0
goalLeftFoot              0
goalHead                  0
dribbleLost               0
dribbleTotal              0
duelAerialLost            0
duelAerialTotal           0
duelAerialWon             0
passTotal                 0
passLongBallAccurate      0
passLongBallInaccurate    0
shortPassAccurate         0
shortPassInaccurate       0
passCornerAccurate        0
passCornerInaccurate      0
passFreekickAccurate      0
passFreekickInaccurate    0
passCrossInaccurate       0
passCrossAccurate         0
keyPassLong               0
keyPassShort        

In [27]:
train_df_ws[train_df_ws.NewName == 'E. Hazard']

,NewName,MarketValue,ClubInvolved,League,MainPosition,LeagueInvolved,CorrMarketValue,ID,Name,Age,...,keyPassFreekick,keyPassThrowin,keyPassOther,assistCross,assistCorner,assistThroughball,assistFreekick,assistThrowin,assistOther,assist
4211,E. Hazard,33.0,LOSC Lille,English Premier League,Forward,French Ligue 1,16.500000,183277,E. Hazard,21,...,0.263158,0.0,1.578947,0.236842,0.052632,0.078947,0.052632,0.0,0.105263,0.421053
11568,E. Hazard,150.0,Chelsea FC,Spain Primera Division,Forward,English Premier League,42.857143,183277,E. Hazard,28,...,0.162162,0.0,1.702703,0.108108,0.054054,0.027027,0.027027,0.0,0.270270,0.405405


In [34]:
train_df_ws.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/train_ws_df.xlsx")